**Demonstration of Batch Scoring with scoring endpoint API from Azure Machine Learning**

In [27]:
# Import necessary libraries.
import pandas as pd
import json
import urllib.request
import os
import ssl
from pandas import json_normalize
import numpy as np

In [2]:
# Import test dataset that does not include in model training.
df = pd.read_csv('./data/WA_Fn-UseC_-Telco-Customer-Churn_R2_Test.csv')

In [3]:
# Display header of test dataset.
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1122-JWTJW,1,0,1,1,1,1,0,2,0,...,0,0,0,0,0,1,3,70.65,70.65,1
1,9710-NJERN,0,0,0,0,39,1,0,0,2,...,2,2,2,2,2,0,3,20.15,826.00,0
2,9837-FWLCH,1,0,1,1,12,1,0,0,2,...,2,2,2,2,0,1,2,19.20,239.00,0
3,1699-HPSBG,1,0,0,0,12,1,0,1,0,...,0,1,1,0,1,1,2,59.80,727.80,1
4,7203-OYKCT,1,0,0,0,72,1,1,2,0,...,1,0,1,1,1,1,2,104.95,7544.30,0


In [4]:
# Display info of test dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        21 non-null     object 
 1   gender            21 non-null     int64  
 2   SeniorCitizen     21 non-null     int64  
 3   Partner           21 non-null     int64  
 4   Dependents        21 non-null     int64  
 5   Tenure            21 non-null     int64  
 6   PhoneService      21 non-null     int64  
 7   MultipleLines     21 non-null     int64  
 8   InternetService   21 non-null     int64  
 9   OnlineSecurity    21 non-null     int64  
 10  OnlineBackup      21 non-null     int64  
 11  DeviceProtection  21 non-null     int64  
 12  TechSupport       21 non-null     int64  
 13  StreamingTV       21 non-null     int64  
 14  StreamingMovies   21 non-null     int64  
 15  Contract          21 non-null     int64  
 16  PaperlessBilling  21 non-null     int64  
 17 

In [5]:
# Duplicate test dataset for model scoring.
df_to_score = df.copy()

In [6]:
# Drop unnecessary columns for model scoring.
df_to_score.drop('customerID', axis=1, inplace=True)
df_to_score.drop('Churn', axis=1, inplace=True)

In [7]:
# Display header of test dataset for scoring.
df_to_score.head()

,gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,1,1,1,1,0,2,0,0,0,0,0,0,0,1,3,70.65,70.65
1,0,0,0,0,39,1,0,0,2,2,2,2,2,2,2,0,3,20.15,826.00
2,1,0,1,1,12,1,0,0,2,2,2,2,2,2,0,1,2,19.20,239.00
3,1,0,0,0,12,1,0,1,0,0,0,1,1,0,1,1,2,59.80,727.80
4,1,0,0,0,72,1,1,2,0,1,1,0,1,1,1,1,2,104.95,7544.30


In [9]:
# Display info of test dataset for scoring.

df_to_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            21 non-null     int64  
 1   SeniorCitizen     21 non-null     int64  
 2   Partner           21 non-null     int64  
 3   Dependents        21 non-null     int64  
 4   Tenure            21 non-null     int64  
 5   PhoneService      21 non-null     int64  
 6   MultipleLines     21 non-null     int64  
 7   InternetService   21 non-null     int64  
 8   OnlineSecurity    21 non-null     int64  
 9   OnlineBackup      21 non-null     int64  
 10  DeviceProtection  21 non-null     int64  
 11  TechSupport       21 non-null     int64  
 12  StreamingTV       21 non-null     int64  
 13  StreamingMovies   21 non-null     int64  
 14  Contract          21 non-null     int64  
 15  PaperlessBilling  21 non-null     int64  
 16  PaymentMethod     21 non-null     int64  
 17 

In [10]:
# Convert pandas dataframe of test dataset (scoring) to json format.
posts = json.loads('{"Inputs":' + '{"data":' + df_to_score.to_json(orient='records', date_format='iso') + '}' + '}')

In [11]:
# Display json result in raw format.
print(posts)

{'Inputs': {'data': [{'gender': 1, 'SeniorCitizen': 0, 'Partner': 1, 'Dependents': 1, 'Tenure': 1, 'PhoneService': 1, 'MultipleLines': 0, 'InternetService': 2, 'OnlineSecurity': 0, 'OnlineBackup': 0, 'DeviceProtection': 0, 'TechSupport': 0, 'StreamingTV': 0, 'StreamingMovies': 0, 'Contract': 0, 'PaperlessBilling': 1, 'PaymentMethod': 3, 'MonthlyCharges': 70.65, 'TotalCharges': 70.65}, {'gender': 0, 'SeniorCitizen': 0, 'Partner': 0, 'Dependents': 0, 'Tenure': 39, 'PhoneService': 1, 'MultipleLines': 0, 'InternetService': 0, 'OnlineSecurity': 2, 'OnlineBackup': 2, 'DeviceProtection': 2, 'TechSupport': 2, 'StreamingTV': 2, 'StreamingMovies': 2, 'Contract': 2, 'PaperlessBilling': 0, 'PaymentMethod': 3, 'MonthlyCharges': 20.15, 'TotalCharges': 826.0}, {'gender': 1, 'SeniorCitizen': 0, 'Partner': 1, 'Dependents': 1, 'Tenure': 12, 'PhoneService': 1, 'MultipleLines': 0, 'InternetService': 0, 'OnlineSecurity': 2, 'OnlineBackup': 2, 'DeviceProtection': 2, 'TechSupport': 2, 'StreamingTV': 2, 'Stre

In [12]:
# Convert raw json result to consumable json format.
json_string = json.dumps(posts)

In [13]:
# Display json result in consumable format.
print(json_string)

{"Inputs": {"data": [{"gender": 1, "SeniorCitizen": 0, "Partner": 1, "Dependents": 1, "Tenure": 1, "PhoneService": 1, "MultipleLines": 0, "InternetService": 2, "OnlineSecurity": 0, "OnlineBackup": 0, "DeviceProtection": 0, "TechSupport": 0, "StreamingTV": 0, "StreamingMovies": 0, "Contract": 0, "PaperlessBilling": 1, "PaymentMethod": 3, "MonthlyCharges": 70.65, "TotalCharges": 70.65}, {"gender": 0, "SeniorCitizen": 0, "Partner": 0, "Dependents": 0, "Tenure": 39, "PhoneService": 1, "MultipleLines": 0, "InternetService": 0, "OnlineSecurity": 2, "OnlineBackup": 2, "DeviceProtection": 2, "TechSupport": 2, "StreamingTV": 2, "StreamingMovies": 2, "Contract": 2, "PaperlessBilling": 0, "PaymentMethod": 3, "MonthlyCharges": 20.15, "TotalCharges": 826.0}, {"gender": 1, "SeniorCitizen": 0, "Partner": 1, "Dependents": 1, "Tenure": 12, "PhoneService": 1, "MultipleLines": 0, "InternetService": 0, "OnlineSecurity": 2, "OnlineBackup": 2, "DeviceProtection": 2, "TechSupport": 2, "StreamingTV": 2, "Stre

In [68]:
# Define function to handle self signed SSL.
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

In [15]:
# Prepare post request body from test dataset for scoring.
body = str.encode(json.dumps(posts))

In [16]:
# Define necessary variables for calling scoring endpoint API from Azure Machine Learning.
# url = 'http://PLEASE-ENTER-YOUR-OWNED-AML-ACI-ENDPOINT-NAME.southeastasia.azurecontainer.io/score'
# api_key = 'PLEASE-ENTER-YOUR-OWN-API-KEY' # Replace this with the API key for the web service
url = 'http://PLEASE-ENTER-YOUR-OWNED-AML-ACI-ENDPOINT-NAME.southeastasia.azurecontainer.io/score'
api_key = 'PLEASE-ENTER-YOUR-OWN-API-KEY' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib.request.Request(url, body, headers)

In [17]:
# Perform scoring with scoring endpoint API from Azure Machine Learning.
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'{"Results": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]}'


In [18]:
# Convert the scoring result from raw to json format.
temp_json = json.loads(result)

In [19]:
# Display the json scoring result.
temp_json

{'Results': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]}

In [20]:
# Convert json scoring result to Pandas dataframe. 
temp_json_df = json_normalize(temp_json, "Results")

In [21]:
# Rename column name to "Churn_Predict".
temp_json_df.rename(columns = {0:"Churn_Predict"}, inplace=True)

In [22]:
# Display info of scoring result dataframe.
temp_json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Churn_Predict  21 non-null     int64
dtypes: int64(1)
memory usage: 296.0 bytes


In [23]:
# Display scoring result dataframe.
temp_json_df

,Churn_Predict
0,1
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [24]:
# Merge scoring result dataframe into original dataframe. 
result_df = pd.concat([df, temp_json_df], axis=1, join="inner")

In [25]:
# Display the merged result.
result_df

,customerID,gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Churn_Predict
0,1122-JWTJW,1,0,1,1,1,1,0,2,0,...,0,0,0,0,1,3,70.65,70.65,1,1
1,9710-NJERN,0,0,0,0,39,1,0,0,2,...,2,2,2,2,0,3,20.15,826.00,0,0
2,9837-FWLCH,1,0,1,1,12,1,0,0,2,...,2,2,2,0,1,2,19.20,239.00,0,0
3,1699-HPSBG,1,0,0,0,12,1,0,1,0,...,1,1,0,1,1,2,59.80,727.80,1,0
4,7203-OYKCT,1,0,0,0,72,1,1,2,0,...,0,1,1,1,1,2,104.95,7544.30,0,0
5,1035-IPQPU,0,1,1,0,63,1,1,2,0,...,0,1,1,0,1,2,103.50,6479.40,0,0
6,7398-LXGYX,1,0,1,0,44,1,1,2,1,...,0,0,0,0,1,1,84.80,3626.35,0,0
7,2823-LKABH,0,0,0,0,18,1,1,2,0,...,1,0,1,0,1,0,95.05,1679.40,0,0
8,8775-CEBBJ,0,0,0,0,9,1,0,1,0,...,0,0,0,0,1,0,44.20,403.35,1,0
9,0550-DCXLH,1,0,0,0,13,1,0,1,0,...,1,1,1,0,0,3,73.35,931.55,0,0


In [29]:
# Using numpy to compare the values from columns "Churn" and "Churn_Predict". "Churn" is the ground truth value.
comparison_result = np.where(result_df["Churn"] == result_df["Churn_Predict"], True, False)

In [33]:
# Make a count of comparison result.
counts = np.unique(comparison_result, return_counts=True)

In [39]:
# Display the count.
counts

(array([False,  True]), array([ 4, 17], dtype=int64))

In [44]:
# Select the values from counts.
counts_value = counts[1]

In [67]:
# Display the accuracy rate of model scoring.
print("Churn Prediction Accuracy is " + "{:.2f}".format(counts_value[1]/(counts_value[0]+counts_value[1])*100) + "%")

Churn Prediction Accuracy is 80.95%
